In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import sys


/home/jsdavis/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
import torch

num_gpus = torch.cuda.device_count()
print(f"Available GPUs: {num_gpus}")

for i in range(num_gpus):
    print(f"Device ID: {i}, GPU Name: {torch.cuda.get_device_name(i)}")


Available GPUs: 0


In [3]:
from accelerate import Accelerator
accelerator = Accelerator()
device = accelerator.device

In [4]:
# model_path = 'HuggingFaceH4/zephyr-7b-alpha'
model_path='/media/d1/huggingface.co/models/meta-llama/Llama-2-7b-chat-hf/'
# model_path='meta-llama/Llama-2-7b-chat-hf' 

In [5]:
prompt = "Please list 17 U.S. politicians that have been active since 1980. Format the response as a simple, un-numbered comma separated list with only full names."

In [6]:
prompt = "List 20 famous U.S. politicians that also have a famous blood relative in politics. List only their names. Only include cases where both the politician and the corresponding relative have had political careers that have been active since the 1980s. Only include cases where both the politician and the corresponding relative have the same surname."

In [7]:
prompt

'List 20 famous U.S. politicians that also have a famous blood relative in politics. List only their names. Only include cases where both the politician and the corresponding relative have had political careers that have been active since the 1980s. Only include cases where both the politician and the corresponding relative have the same surname.'

In [54]:
# from entity_extraction import EntityExtraction

# def get_political_names(prompt, minResults=5, retryCnt=3):
#     model_input = tokenizer(prompt, return_tensors="pt").to(device)
#     # Perform inference
#     model.eval()
#     with torch.no_grad():
#         response_id = model.generate(**model_input, max_length=model_input.input_ids.size(1) + 500, temperature=0.4, do_sample=True)

#     # Extract and display the response
#     response_text = tokenizer.decode(response_id[0], skip_special_tokens=True)[len(tokenizer.decode(model_input.input_ids[0], skip_special_tokens=True)):]
#     extractor = EntityExtraction()

#     names = []
#     while True:
#         names = extractor.get_full_names(response_text)
#         if len(names) < minResults and retryCnt > 0:
#             retryCnt =- 1
#             continue
#         break

#     return names


In [8]:
import sys
import os

current = os.path.dirname(os.path.realpath('query-people.py'))
parent = os.path.dirname(current)
sys.path.append(parent)

import query_llm 

llmAccessor = query_llm.QueryLLM(model_path)
names = llmAccessor.get_political_names(prompt)
print("names: ", names)    


Preparing to query LLM....
Query attempt number 0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

names:  ['Bill Clinton', 'Hillary Clinton', 'John F. Kennedy', 'Robert F. Kennedy', 'Barack Obama', 'Michelle Obama', 'George W. Bush', 'Jeb Bush', 'Jimmy Carter', 'Rosalynn Carter', 'Ronald Reagan', 'Nancy Reagan', 'Richard Nixon', 'Tricia Nixon Cox', 'Gerald Ford', 'Betty Ford', 'Dwight D. Eisenhower', 'John Eisenhower', 'Franklin D. Roosevelt', 'James Roosevelt', 'Harry S. Truman', 'Margaret Truman', 'Thomas E. Dewey', 'John Dewey', 'Adlai Stevenson', 'Adlai Stevenson II', 'Hubert Humphrey', 'Hubert Humphrey Jr.', 'Lyndon B. Johnson', 'Lyndon B. Johnson Jr.', 'Robert Taft', 'Taft Jr.', 'Strom Thurmond', 'Strom Thurmond Jr.', 'Barry Goldwater', 'Barry Goldwater Jr.', 'John McCain', 'John McCain Jr.', 'Mitt Romney', 'Mitt Romney Jr']


In [19]:
prompt = "For each politicians returned in the list above, please list a corresponding blood relative that is also in politics."

In [20]:
model_input = tokenizer(prompt, return_tensors="pt").to(device)

# Perform inference
model.eval()
with torch.no_grad():
    response_id = model.generate(**model_input, max_length=model_input.input_ids.size(1) + 500, temperature=0.4, do_sample=True)

# Extract and display the response
response_text = tokenizer.decode(response_id[0], skip_special_tokens=True)[len(tokenizer.decode(model_input.input_ids[0], skip_special_tokens=True)):]
print("User Prompt:")
print(prompt)
print("\nResponse:")
print(response_text.strip())

User Prompt:
For each politicians returned in the list above, please list a corresponding blood relative that is also in politics.

Response:
For example, for Angela Merkel, her brother Horst Merkel is also in politics.

Note: I will only consider blood relatives, not spouses or partners.
